# Computación Científica II
## Laboratorio  #4
    
   Gabriela González Toledo      gabriela.gonzalez@alumnos.usm.cl  201173017-8
   
   Ian Zamorano Escobedo         Ian.zamorano.12@sansano.usm.cl    201273018-k

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt

import timeit
from math import ceil, floor
from sympy import *
from sympy.abc import *
from scipy.special import gamma
from matplotlib import cm

#%load_ext line_profiler

### Introducción:

En este laboratorio trabajaremos en la resolución de la ecuación de onda (EDP Hiperbólica) utilizando distintas condiciones
de borde y una velocidad constante. A medida que resolvamos y grafiquemos nuestras soluciones veremos cual es el comportamiento al cambiar las condiciones de borde. También veremos que sucede graficamente al acercarnos a la inestabilidad del método variando el tamaño de los intervalos de tiempo.

### Ley de Ampere

#### 1. Ecuación característica:

$$
\vec{\nabla} \times \vec{B} =\mu_0 \vec{J} + \mu_0 \epsilon_0 \cfrac{\partial \vec{E}}{\partial t} 
$$

$$
\vec{J} = \vec{0}
$$

$$
\vec{\nabla} \times \vec{B} = \mu_0 \epsilon_0 \cfrac{\partial \vec{E}}{\partial t} \qquad / \; \vec{\nabla} \times
$$

$$
\vec{\nabla} \times (\vec{\nabla} \times \vec{B}) = \vec{\nabla} \times \mu_0 \epsilon_0 \cfrac{\partial \vec{E}}{\partial t}
$$

$$
\vec{\nabla} (\vec{\nabla} \vec{B}) - \vec{\nabla}^{2} \vec{B} = \vec{\nabla} \times \mu_0 \epsilon_0 \cfrac{\partial \vec{E}}{\partial t}
$$

$$
Por ~ Gauss (\vec{\nabla} \cdot \vec{B} = 0)
$$

$$
\therefore - \vec{\nabla}^{2} \vec{B} = \vec{\nabla} \times \mu_0 \epsilon_0 \cfrac{\partial \vec{E}}{\partial t}
$$

$$
Por ~ Faraday-Lenz \left( \vec{\nabla} \times \vec{E} = - \cfrac{\partial B }{\partial t}\right)
$$

$$
 - \vec{\nabla}^{2} \vec{B} = - \mu_0 \epsilon_0 \cfrac{\partial^2 \vec{B}}{\partial t^2}
$$

$$
 \vec{\nabla}^{2} \vec{B} = \mu_0 \epsilon_0 \cfrac{\partial^2 \vec{B}}{\partial t^2}
$$

$$
\vec{B}_{xx} + \vec{B}_{yy}= \mu_0 \epsilon_0 \vec{B}_{tt}
$$


#### 2. Significado fisico:

$\epsilon_0$ : Fisicamente esta constante representa la permitividad eléctrica del vacío.

$\mu_0 $ : Esta constante corresponde a la permeabilidad magnética del vacío.

#### 3.  Discretizacion de ecuacion de onda

$$
\vec{B} \rightarrow u
$$

$$
u_{i,j,k} = B(x_i,y_j,t_k) = u(i\Delta x, j\Delta y, k \Delta t) \qquad i,j,k \in \{1,2,3, \dots , N\}
$$

$$
u_{xx} = \cfrac{u_{i-1,j,k} + 2 u_{i,j,k} + u_{i+1,j,k}}{\Delta x^{2}}
$$

$$
u_{yy} = \cfrac{u_{i,j-1,k} + 2 u_{i,j,k} + u_{i,j+1,k}}{\Delta y^{2}}
$$

$$
u_{tt} = \cfrac{u_{i,j,k-1} + 2 u_{i,j,k} + u_{i+1,j,k+1}}{\Delta t^{2}}
$$

$$
u_{xx} + u_{yy} = \mu_0 \epsilon_0 u_{tt}
$$


#### 4. Analisis de sensibilidad

analizando la formula general dado las dimensiones del problema tenemos lo siguiente:

$$
C = \frac{u_x\Delta t}{\Delta x} + \frac{u_y\Delta t}{\Delta y} \leq C_{Max}
$$
Utilizando $C_{Max} = \cfrac{1}{8 \cdot 10^{8}}$ por el coeficiente de $ u_{tt} $ y los $u$ sub $x$ e $y$ equivalen a 1 dado sus resppectivos coeficientes.

$$
C = \frac{\Delta t}{\Delta x} + \frac{\Delta t}{\Delta y} \leq \cfrac{1}{8 \cdot 10^{8}}
$$

lo que significa que $\Delta t$ debe ser muy pequeño respecto a los $\Delta x$ y $\Delta y$ suponiendo que nuestros $\Delta $ siempre seran menores que 1. 

#### 5. Resolucion del problema.

Dado la forma de nuestro stencil, una cruz tridimencional, debemos contar ahora con planos de valoras iniciales, formando un prisma sin una cara basal, donde esa cara basal que falta representa el tiempo T final que no esta definido, por lo que la region de resolucion de la edp es dentro de este prisma.

La resolucion de esta se ira haciendo por capas, tomando cada capa como un paso de t, inicialmente se utiliza algo similar al caso de ondas en 2D, dado que el stencil nos pide un paso -1, pero ese se logra despejado de la funcion para calcular la misma. ahora que tenemos nuestro primer paso del stencil podemos seguir avanzando normalmente con la discretizacion propuesta, dado que tenemos todos los puntos necesarios para calcular el paso +1 de t.

Finalmente llegaremos a cierto paso T final, con esto tendremos un prisma formado por la malla representativa de la discretizacion.

### Analisis práctico

#### 1. implementacion pde_solver

In [ ]:
def pde_solver(h, k, t_max, c, f, g, period=True, alpha=1, beta=1, l=0 , r=0):
    Nx = int(2/h) + 1 
    Nt = int(t_max/k) + 1
    x = np.linspace(-1,1,Nx)
    t = np.linspace(0,t_max, Nt)
    u = np.zeros((Nx,Nt))
    i = np.arange(1,Nx-1)
    i = map(int, i)
    i = np.array(i)
    dx = h
    dt = k
    sigma_2 = (c(x[i])*dt/dx)**2
    print "sigma cuadrado : ", sigma_2
    for j, j_t in enumerate(t):
        if j == 0:
            u[:,0]= f(x)
        elif j==1:
            u[ i, j] = 0.5*sigma_2*u[i+1,j-1] + (1 - sigma_2)*u[i,j-1] + 0.5*sigma_2*u[i-1, j-1] + dt*g(x[i])
            if period:
                u[ 0, j] = 0.5*sigma_2*u[1,j-1] + (1 - sigma_2)*u[0,j-1] + 0.5*sigma_2*u[-2, j-1] + dt*g(x[0])
                u[-1, j] = u[0 , j]
            else:
                u[-1, j] = (dx*r(j_t)+ (1 -beta)*u[-2,j])/(beta*(dx-1)+1)
                u[ 0, j] = (dx*l(j_t)-(1-alpha)*u[1,j])/(alpha*(dx+1)-1)
        else:
            u[i,j] = sigma_2*u[i+1,j-1]+(2-2*sigma_2)*u[i,j-1]+sigma_2*u[i-1,j-1]-u[i,j-2]
            if period:
                u[ 0, j] = 0.5*sigma_2*u[1,j-1] + (1 - sigma_2)*u[0,j-1] + 0.5*sigma_2*u[-2, j-1] + dt*g(x[0])
                u[-1, j] = u[0 , j]
            else:
                u[-1, j] = (dx*r(j_t)+ (1 -beta)*u[-2,j])/(beta*(dx-1)+1)
                u[ 0, j] = (dx*l(j_t)-(1-alpha)*u[1,j])/(alpha*(dx+1)-1)
    return x , t , np.transpose(u)

#### 2.Cálculo longitud de onda y tiempo con 500 puntos

In [ ]:
#datos
f = lambda x: 100*np.sin(x**2)-100*np.sin(1)
g = lambda x: 200*x*np.sin(x**2)
c = lambda x: 1
l = lambda t: 0
r = lambda t: 0


In [ ]:
#funcion para graficar la superficie
def graficar_superficie(x,t,u,titulo):
    fig = plt.figure()
    ax = fig.gca(projection='3d',title = titulo, xlabel = "x", ylabel = "t", zlabel="u")
    X, T = np.meshgrid(x, t)
    surf = ax.plot_surface(X, T, u, rstride=1,cmap=cm.coolwarm,cstride=1,linewidth=0, antialiased=False)
    print "dimension de la matriz: ",X.shape
    bul = (u.shape == X.shape and T.shape == u.shape)
    plt.show()

In [ ]:
# conocer el indice de cierto dato
def index_array(A,x):
    for i in range(A.shape[0]):
        if x == A[i]:
            return i
    return None

In [ ]:
# conocer el indice de cierto dato
def index_array_tol(A,x,tol):
    for i in range(A.shape[0]):
        if A[i] - tol < x and x > A[i] + tol:
            return i
    return None

In [ ]:
def periodo(u,delta_t):
    arreglo = u[:][u.shape[1]/(u.shape[1]/2)]
    maximo = arreglo.max()
    index = arreglo.argmax()
    index_2 = index_array(arreglo[index+1:], maximo)
    if index_2 != None:
        return (index_2 - index)*delta_t
    else:
        index_2 = index_array_tol(arreglo[index+1:], maximo,0.5)
        if index_2 != None:
            return abs((index_2 - index)*delta_t)
        else:
            return "incalculable para esta presicion e intervalo"
        

In [ ]:
def longitud_onda(u,delta_x):
    arreglo = u[0][:]
    minimo = arreglo.min()
    index = arreglo.argmin()
    index_2 = index_array(arreglo[index+1:], minimo)
    if index_2 != None:
        return (index_2 - index)*delta_x
    else:
        index_2 = index_array_tol(arreglo[index+1:], minimo,0.5)
        if index_2 != None:
            return abs((index_2 - index)*delta_t)
        else:
            return "incalculable para esta presicion e intervalo"

In [ ]:
%matplotlib inline
# Dirichlet
x, t, u = pde_solver(2.0/10, 10.0/50, 10, c, f, g, False , 1, 1, l, r)
titulo = "Dirichlet con h = " + str(2.0/10)+"y k = " + str(10/50)
print "el periodo corresponde a : " + str(periodo(u,10.0/50)) 
print "la longitud de onda corresponde a : " + str(longitud_onda(u,2.0/10)) 
graficar_superficie(x,t,u,titulo)

In [ ]:
%matplotlib inline
# Neumann
x, t, u = pde_solver(2.0/10, 4.0/50, 4, c, f, g, False , 0, 0, l, r)
titulo = "Dirichlet con h = " + str(2.0/10)+"y k = " + str(10/50)
print "el periodo corresponde a : " + str(periodo(u,10.0/50)) 
print "la longitud de onda corresponde a : " + str(longitud_onda(u,2.0/10)) 
graficar_superficie(x,t,u,titulo)

In [ ]:
%matplotlib inline
# Mixed, alpha = 1 , beta = 0
h = n[m]
x, t, u = pde_solver(h,h/10, 4, c, f, g, False , 0, 1, l, r)
titulo = "Mixed: alpha = 1 , beta = 0 con h = " + str(h)+"y k = " + str(h/10)
print "el periodo corresponde a : " + str(periodo(u,10.0/50)) 
print "la longitud de onda corresponde a : " + str(longitud_onda(u,2.0/10)) 
graficar_superficie(x,t,u,titulo)



# Mixed, alpha = 0 , beta = 1
x, t, u = pde_solver(2.0/10, 4.0/50, 4, c, f, g, False , 1, 0, l, r)
titulo = "Mixed: alpha = 0 , beta = 1 con h = " + str(h)+"y k = " + str(h/10)
print "el periodo corresponde a : " + str(periodo(u,10.0/50)) 
print "la longitud de onda corresponde a : " + str(longitud_onda(u,2.0/10)) 
graficar_superficie(x,t,u,titulo)

In [ ]:
#Periodica
x, t, u = pde_solver(2.0/10, 10.0/50, 10, c, f, g)
titulo = "Periodica con h = " + str(h)+"y k = " + str(h/10)
print "el periodo corresponde a : " + str(periodo(u,10.0/50)) 
print "la longitud de onda corresponde a : " + str(longitud_onda(u,2.0/10)) 
graficar_superficie(x,t,u,titulo)

#### 3.Cálculo longitud de onda y tiempo con malla fina

In [ ]:
n = ([2.0/10,2.0/30,2.0/50,2.0/70,2.0/100])

In [ ]:
%matplotlib inline
# Dirichlet
for m in range(len(n)):
    h = n[m]
    x, t, u = pde_solver(h/10, h/20, 10, c, f, g, False , 1, 1, l, r)
    print "----------pde_solve listo-------------"
    titulo = "Dirichlet con h = " + str(h)+"y k = " + str(h/10)
    graficar_superficie(x,t,u,titulo)
print "........... FIN DE GRAFICOS .............."


In [ ]:
%matplotlib inline
# Neumann
for m in range(len(n)):
    h = n[m]
    x, t, u = pde_solver(h,h/10, 4, c, f, g, False , 0, 0, l, r)
    titulo = "Neumann con h = " + str(h)+"y k = " + str(h/10)
    graficar_superficie(x,t,u,titulo)
print "........... FIN DE GRAFICOS .............."

In [ ]:
%matplotlib inline
# Mixed, alpha = 1 , beta = 0
for m in range(len(n)):
    h = n[m]
    x, t, u = pde_solver(h,h/10, 4, c, f, g, False , 0, 1, l, r)
    titulo = "Mixed: alpha = 1 , beta = 0 con h = " + str(h)+"y k = " + str(h/10)
    graficar_superficie(x,t,u,titulo)



# Mixed, alpha = 0 , beta = 1
for m in range(len(n)):
    h = n[m]
    x, t, u = pde_solver(h, h/10, 4, c, f, g, False , 1, 0, l, r)
    titulo = "Mixed: alpha = 0 , beta = 1 con h = " + str(h)+"y k = " + str(h/10)
    graficar_superficie(x,t,u,titulo)
print "........... FIN DE GRAFICOS .............."

In [ ]:

# Periodica
for m in range(len(n)):
    h = n[m]
    x, t, u = pde_solver(h, h/10, 10, c, f, g)
    titulo = "Periodica con h = " + str(h)+"y k = " + str(h/10)
    graficar_superficie(x,t,u,titulo)
print "........... FIN DE GRAFICOS .............."

#### 4. Inestabilidad

In [ ]:
h = 2.0/70

In [ ]:
%matplotlib inline
# Dirichlet Inestable
x, t, u = pde_solver(h/10, h*2, 10, c, f, g, False , 1, 1, l, r)
titulo = "Dirichlet inestable con h = " + str(h)+"y k = " + str(h*2)
graficar_superficie(x,t,u,titulo)

In [ ]:
%matplotlib inline
# Neumann inestable
x, t, u = pde_solver(h,h*2, 4, c, f, g, False , 0, 0, l, r)
titulo = "Neumann inestable con h = " + str(h)+"y k = " + str(h*2)
graficar_superficie(x,t,u,titulo)

In [ ]:
%matplotlib inline
# Mixed inestable, alpha = 1 , beta = 0

x, t, u = pde_solver(h,h/10, 4, c, f, g, False , 0, 1, l, r)
titulo = "Mixed inestable: alpha = 1 , beta = 0 con h = " + str(h)+"y k = " + str(h/10)
graficar_superficie(x,t,u,titulo)


# Mixed inestable, alpha = 0 , beta = 1

x, t, u = pde_solver(h, h*2, 4, c, f, g, False , 1, 0, l, r)
titulo = "Mixed Inestable: alpha = 0 , beta = 1 con h = " + str(h)+"y k = " + str(h/10)
graficar_superficie(x,t,u,titulo)


In [ ]:
# Periodica
for m in range(len(n)):
    h = n[m]
    x, t, u = pde_solver(h, h/10, 10, c, f, g)
    titulo = "Periodica con h = " + str(h)+"y k = " + str(h/10)
    graficar_superficie(x,t,u,titulo)

### Conclusión:

### Referencias

http://matplotlib.org/examples/mplot3d/surface3d_demo.html

https://es.wikipedia.org/wiki/Prisma_(geometr%C3%ADa)
